In [1]:
print('Importing...')

import glob
import os
import numpy as np
import time as systime
import re 
import textgrid as tg

from bokeh.plotting import show
from bokeh.io import output_notebook
from bokeh.plotting import figure
from bokeh.layouts import gridplot
from ipyparallel import Client
import _pickle as pickle

import sys

print('success')

Importing...
success


- we need to start with a top level string 
- do some edit operations to get bottom level
- have each bottom level type index a 3-state hmm
- for each 3 state hmm
	- sample some transitions etc 
	- have each state index a GMM
	- each GMM generate a 2-dimensional vector 
    
- input: top level string
- parameters we need to specify: 
	- edit operation parameters
		- ins_top (1)
		- ins_bot (n)
		- sub (n) 
	- hmm 
		- initial (3)
		- transition (3x3)
		- emission (3xm)
    - GMM
		- num of components 
		- mean, variance for each component 
		- mixture proportions 

for simplicity, let's make all top level strings 10 characters long.
let's have a 4 character alphabet 

In [2]:
top_alphabet = [0,1,2,3]
print('success')

success


 let's have 1 more bottom level PLU



In [3]:
bot_alphabet = [0,1,2,3,4]
print('success')

success


Now we get some random sequences of top letters that represent a top alphabet

In [4]:
top_strings = np.random.choice(top_alphabet, (100, 10))
print('success')

success


Now we define prob distributions over ins, sub, del, and sample edit operations for each top-level PLU

In [5]:
# Dirichlet parameters for distribution over edit ops for each of the top PLUs 
# (plus distribution over final insert bottoms)
# (insert_top prob) (insert_bottom probs) (sub probs)
ops_priors = [
    [1,1,1,1,1,1,3,2,1,1,1],
    [2,1,1,1,1,1,1,3,2,1,1],
    [3,1,1,1,1,1,1,1,3,1,1],
    [1,1,1,2,2]
]

# Draw a sample from each Dirichlet to get the distribution
ops_dists = [np.random.dirichlet(i) for i in ops_priors]

# ins_top = [2]
# ins_bot = [.5,1,.5,4,2]
# sub = [[100, 3, 4, 1],
#       [.5, 100, 2, 2],
#       [2, .75, 100, 3],
#       [3,2,1,100]]


# full_dir = np.array(ins_top + ins_bot + sub[0] + sub[1] + sub[2] + sub[3], dtype=np.float64)
# # normalize 
# full_dir = full_dir/np.sum(full_dir)
# print(full_dir)

print('success')


success


In [6]:
bottom_strings = []
i = 0
while i < len(top_strings):
    ts = top_strings[i]
    j = 0
    bs = []
    while j < len(ts):
#         sample some edit operation
#         eo = np.argmax(np.random.multinomial(1, full_dir))
        eo = np.argmax(np.random.multinomial(1, ops_dists[ts[j]]))
        if eo == 0:
#             insert top, do nothing
            j+=1
        elif eo >= 1 and eo <= 5:
            bc = bot_alphabet[eo-1]
            bs.append(bc)
            continue
        else:
#             sub
#             bc = bot_alphabet[(eo-5)%4]
            bc = bot_alphabet[(eo-6)] 
            bs.append(bc)
            j+=1
    i+=1
    bottom_strings.append(bs)

print(bottom_strings)
print(len(bottom_strings[0]))



[[3, 3, 2, 1, 0, 1, 0, 1, 3, 0, 3, 3, 0, 2, 3, 3, 2, 3, 2, 3, 3, 2, 2, 3, 2, 3, 0, 1, 0], [1, 4, 2, 1, 4, 2, 2, 3, 3, 2, 3, 2, 3, 2, 2, 3, 3, 0, 3, 2, 4, 4, 2, 4, 1], [0, 4, 0, 2, 3, 0, 1, 4, 0, 4, 0, 2, 1, 3, 2, 2, 4, 4], [1, 1, 1, 2, 1, 3, 3, 3, 3, 2, 2, 3, 3, 2, 2, 0, 0, 1, 4, 2, 2, 3, 0, 3, 3, 2, 3, 2, 2, 3, 2, 3, 3, 3, 2, 3, 3, 3, 2, 2, 3, 0, 2, 3, 3, 2, 2, 3, 0, 2, 3, 2], [2, 0, 3, 3, 2, 3, 3, 3, 3, 0, 0, 1, 3, 0, 2, 3, 4, 1, 3, 2, 2, 2], [3, 3, 3, 3, 2, 3, 3, 2, 3, 3, 2, 3, 0, 3, 3, 3, 3, 1, 4, 2, 0, 2, 2, 0, 2, 2, 3, 3, 2, 0, 2, 2, 2, 3, 2, 0], [3, 1, 1, 4, 1, 4, 2, 3, 4, 0, 3, 3, 0, 3, 2, 2, 3, 3, 2, 3, 3, 2, 2, 2, 2], [2, 3, 2, 2, 2, 3, 2, 3, 3, 2, 2, 3, 2, 3, 1, 4, 4, 1, 3, 3, 4, 0, 4, 0, 0, 4, 2, 2], [2, 1, 1, 0, 2, 4, 0, 1, 4, 2, 3, 1, 0, 3, 3, 2, 3, 4], [2, 2, 3, 2, 4, 2, 3, 3, 2, 3, 3, 3, 2, 2, 2, 3, 3, 3, 2, 2, 2, 3, 2, 2, 2, 3, 0, 3, 0, 0, 2, 2, 2, 3, 3, 3, 2, 2, 2, 2, 3, 2, 2, 3, 0, 2, 3, 3, 2, 3, 3, 2, 3, 2, 3, 3, 3, 3, 3, 3, 2, 0, 3, 3, 3, 3, 2, 3, 2, 0, 3], [0, 2, 

Let's define the HMM/GMM parameters

In [7]:
# transition = [[.5, .5, 0],
#              [0, .5, .5],
#              [.75, 0, .25]]

components = []
# Distribution over components for each bottom PLU state
components.append([[.7,.3],
             [.9,.1],
             [.3,.7]])
components.append([[.3,.7],
             [.5,.5],
             [.8,.2]])
components.append([[.4,.6],
             [.2,.8],
             [.9,.1]])
components.append([[.5,.5],
             [.3,.7],
             [.6,.4]])
components.append([[.1,.9],
             [.6,.4],
             [.7,.3]])


# We need a set of parameters for each component
# for each HMM state for each bottom PLU
# So in this case, we need 2 * 3 * 5 = 30 sets of parameters
component_parameters = [
    [
        [
            [[0,0], [[1,0],[0,1]]], # bottom PLU 0, HMM state 0, component 0
            [[1,1], [[1,0],[0,1]]]  # bottom PLU 0, HMM state 0, component 1
        ],
        [
             [[2,2], [[1,0],[0,1]]], # bottom PLU 0, HMM state 1, component 0
             [[3,3], [[1,0],[0,1]]]  # bottom PLU 0, HMM state 1, component 1
        ],
        [
            [[4,4], [[1,0],[0,1]]], # bottom PLU 0, HMM state 2, component 0
            [[5,5], [[1,0],[0,1]]]  # bottom PLU 0, HMM state 2, component 1
        ]
    ],
    [
        [
            [[10,10], [[1,0],[0,1]]], # bottom PLU 1, HMM state 0, component 0
            [[11,11], [[1,0],[0,1]]]  # bottom PLU 1, HMM state 0, component 1
        ],
        [
             [[12,12], [[1,0],[0,1]]], # bottom PLU 1, HMM state 1, component 0
             [[13,13], [[1,0],[0,1]]]  # bottom PLU 1, HMM state 1, component 1
        ],
        [
            [[14,14], [[1,0],[0,1]]], # bottom PLU 1, HMM state 2, component 0
            [[15,15], [[1,0],[0,1]]]  # bottom PLU 1, HMM state 2, component 1
        ]
    ],
    [        
        [
            [[20,20], [[1,0],[0,1]]], # bottom PLU 2, HMM state 0, component 0
            [[21,21], [[1,0],[0,1]]]  # bottom PLU 2, HMM state 0, component 1
        ],
        [
             [[22,22], [[1,0],[0,1]]], # bottom PLU 2, HMM state 1, component 0
             [[23,23], [[1,0],[0,1]]]  # bottom PLU 2, HMM state 1, component 1
        ],
        [
            [[24,24], [[1,0],[0,1]]], # bottom PLU 2, HMM state 2, component 0
            [[25,25], [[1,0],[0,1]]]  # bottom PLU 2, HMM state 2, component 1
        ]
    ],
    [
        [
            [[30,30], [[1,0],[0,1]]], # bottom PLU 3, HMM state 0, component 0
            [[31,31], [[1,0],[0,1]]]  # bottom PLU 3, HMM state 0, component 1
        ],
        [
             [[32,32], [[1,0],[0,1]]], # bottom PLU 3, HMM state 1, component 0
             [[33,33], [[1,0],[0,1]]]  # bottom PLU 3, HMM state 1, component 1
        ],
        [
            [[34,34], [[1,0],[0,1]]], # bottom PLU 3, HMM state 2, component 0
            [[35,35], [[1,0],[0,1]]]  # bottom PLU 3, HMM state 2, component 1
        ]
    ],
    [
        [
            [[40,40], [[1,0],[0,1]]], # bottom PLU 4, HMM state 0, component 0
            [[41,41], [[1,0],[0,1]]]  # bottom PLU 4, HMM state 0, component 1
        ],
        [
             [[42,42], [[1,0],[0,1]]], # bottom PLU 4, HMM state 1, component 0
             [[43,43], [[1,0],[0,1]]]  # bottom PLU 4, HMM state 1, component 1
        ],
        [
            [[44,44], [[1,0],[0,1]]], # bottom PLU 4, HMM state 2, component 0
            [[45,45], [[1,0],[0,1]]]  # bottom PLU 4, HMM state 2, component 1
        ]
    ]


]

print('success')
# n1 = [[0,0], [[1,0],[0,1]]]
# n2 = [[10, 10], [[1,0],[0,1]]]


success


In [8]:
print('running...')

def sample_HMMGMM(bs):
    mfccs = []
    for i, char in enumerate(bs):
        vec = None
#         if i%10 == 0:
#             print("done with ", i)
#         start_state = np.random.choice([0,1,2], p=init)
        curr_state = 0
#       get the mfcc vector and transition
        while curr_state < 3:
            gmm_component_choice = np.random.choice(2,p=components[char][curr_state])
            vec = np.random.multivariate_normal(*component_parameters[char][curr_state][gmm_component_choice])
#             vec = components[curr_state][0]*np.random.multivariate_normal(*n1) + \
#                     components[curr_state][1]*np.random.multivariate_normal(*n2)
            
            mfccs.append(vec)
#             new_curr_state = np.random.choice([0,1,2], p=transition[curr_state])
            # Transition to next state with 50% probability
            curr_state = np.random.choice([curr_state,curr_state+1],p=[0.5,0.5])
#             if curr_state == 2 and new_curr_state == 0:
    #         get the mfcc vector and transition
#             curr_state = new_curr_state
    return np.array(mfccs)

all_data = []


for i, bs in enumerate(bottom_strings):
    mfccs = sample_HMMGMM(bs)
    all_data.append(mfccs)

print(all_data[0].shape)

print('success')
        
# for i,line in enumerate(all_data):
#     with open("/Users/esteng/ULD/audio/example_2d/sent_{}.fea", "w") as f1:
        
        

running...
(206, 2)
success


In [9]:
def collect_data_stats(data):
    """Job to collect the statistics."""
    # We  re-import this module here because this code will run
    # remotely.
    
    stats_0 = data.shape[0]
    stats_1 = data.sum(axis=0)
    stats_2 = (data**2).sum(axis=0)
    retval = (
        stats_0,
        stats_1,
        stats_2
    )
    return retval

data_stats = list(map(collect_data_stats, all_data))

print('success')

success


In [10]:

def accumulate_stats(data_stats):
    n_frames = data_stats[0][0]
    mean = data_stats[0][1]
    var = data_stats[0][2]
    for stats_0, stats_1, stats_2 in data_stats[1:]:
        n_frames += stats_0
        mean += stats_1
        var += stats_2
    mean /= n_frames
    var = (var / n_frames) - mean**2

    data_stats = {
        'count': n_frames,
        'mean': mean,
        'var': var
    }
    return data_stats

final_data_stats = accumulate_stats(data_stats)

print('success')

success


In [11]:
print("Running...")

import subprocess
import amdtk
num_tops = 4

print("Finished imports in ipython notebook.")

print(len(all_data))
print(len(top_strings))
all_data_and_tops = list(zip(all_data, top_strings))
#print(all_data_and_tops[0])

print("starting engines")
subprocess.Popen(['ipcluster', 'start',' --profile', 'default',' -n', '4', '--daemonize'])
subprocess.Popen(['sleep', '10']).communicate()


rc = Client(profile='default')
dview = rc[:]
print('Connected to', len(dview), 'jobs.')


elbo = []
time = []
def callback(args):
    elbo.append(args['lower_bound'])
    time.append(args['time'])
    print('elbo=' + str(elbo[-1]), 'time=' + str(time[-1]))
 

print("Creating phone loop model...")
conc = 0.1



model = amdtk.PhoneLoopNoisyChannel.create(
    n_units=5,  # number of acoustic units
    n_states=3,   # number of states per unit
    n_comp_per_state=3,   # number of Gaussians per emission
    n_top_units=num_tops, # size of top PLU alphabet
    mean=np.zeros_like(final_data_stats['mean']), 
    var=np.ones_like(final_data_stats['var']),
    max_slip_factor=.05
    #concentration=conc
)


print("Creating VB optimizer...")   
optimizer = amdtk.ToyNoisyChannelOptimizer(
    dview, 
    final_data_stats, 
    args= {'epochs': 4,
     'batch_size': 4,
     'lrate': 0.01,
     'pkl_path': "example_test",
     'log_dir': 'logs'},
    model=model,
    dir_path='dir'
)

print("Running VB optimization...")
begin = systime.time()
print("running with toy data")
print('len(all_data_and_tops[0])')
print(len(all_data_and_tops[0]))
optimizer.run(all_data_and_tops, callback)
end = systime.time()
print("VB optimization took ",end-begin," seconds.")

# fig1 = figure(
#     x_axis_label='time (s)', 
#     y_axis_label='ELBO',
#     width=400, 
#     height=400
# )
# x = np.arange(0, len(elbo), 1)
# fig1.line(x, elbo)
#show(fig1)

print("\nDECODING\n")

date_string = systime.strftime("textgrids_%Y-%m-%d_%H:%M")

# Need to change this according to 
samples_per_sec = 100

all_data_and_tops = list(zip(all_data, top_strings))

for data, tops in all_data_and_tops:

    # Normalize the data
    data_mean = np.mean(data)
    data_var = np.var(data)
    data = (data-data_mean)/np.sqrt(data_var)

    #result = model.decode(data, tops, state_path=False)
    #result_path = model.decode(data, tops, state_path=True)
    (result_intervals, edit_path) = model.decode(data, tops, phone_intervals=True, edit_ops=True)

    print("---")
    print("Phone sequence:")
    print(result_intervals)
    print(edit_path)
    

Running...
INSIDE INIT
SUCCESSFUL IMPORT IN INIT
Finished imports in ipython notebook.
100
100
starting engines
Connected to 4 jobs.
Creating phone loop model...
[-2.92896825 -2.92896825 -2.92896825 -2.92896825 -2.92896825 -2.92896825
 -2.92896825 -2.92896825 -2.92896825 -2.92896825 -2.92896825]
[-2.92896825 -2.92896825 -2.92896825 -2.92896825 -2.92896825 -2.92896825
 -2.92896825 -2.92896825 -2.92896825 -2.92896825 -2.92896825]
[-2.92896825 -2.92896825 -2.92896825 -2.92896825 -2.92896825 -2.92896825
 -2.92896825 -2.92896825 -2.92896825 -2.92896825 -2.92896825]
[-2.92896825 -2.92896825 -2.92896825 -2.92896825 -2.92896825 -2.92896825
 -2.92896825 -2.92896825 -2.92896825 -2.92896825 -2.92896825]
Creating VB optimizer...
importing numpy on engine(s)
importing read_htk from amdtk on engine(s)
importing _pickle on engine(s)
importing os on engine(s)
Running VB optimization...
running with toy data
len(all_data_and_tops[0])
2
[[  2.24632656e-07   1.96571941e-09   5.04962485e-09]
 [  1.1518726

[[  9.02558122e-04   4.04456762e-03   3.17437846e-03]
 [  3.53917225e-04   1.26790919e-03   7.55317823e-03]
 [  2.19104740e-05   1.61422401e-06   1.64767938e-02]
 [  4.23823082e-12   7.01176605e-12   3.97406568e-12]
 [  1.34313266e-12   2.77248770e-11   2.34195471e-13]
 [  1.61952832e-11   3.63129016e-14   4.44178237e-14]
 [  1.72309447e-01   6.31676005e+00   5.83488783e+00]
 [  2.62087726e+01   5.00099901e+00   7.62204715e-01]
 [  8.82642226e+01   2.94425846e+00   5.54522802e+00]
 [  1.28382354e-01   2.67768181e-01   5.34368902e+00]
 [  7.69586085e-02   2.23987736e-02   3.32200656e+01]
 [  6.02162205e-01   9.89906038e-01   2.23078124e+01]
 [  5.88929201e-04   4.59605640e-01   1.20478189e-02]
 [  2.56760312e-03   1.40163643e-02   6.73276791e-01]
 [  1.08587204e-04   1.29835012e-06   7.95204264e-01]]
[[  4.47490870e-03   9.20618693e-04  -1.97018663e-03  -8.68720562e-04
    9.02558122e-04   9.02558122e-04   9.02558122e-04   9.02558122e-04]
 [  1.99067889e-02   3.94295508e-03  -8.97260942

[[  1.37165328e-05   5.78658482e-05   4.58860464e-05]
 [  5.15167854e-06   1.86550842e-05   1.20882524e-04]
 [  2.89543547e-07   2.61743464e-08   2.66420687e-04]
 [  2.82008462e-14   4.64987319e-14   2.56819998e-14]
 [  1.20477877e-14   1.98531531e-13   2.12870857e-15]
 [  1.18991392e-13   3.83336674e-16   5.08423510e-16]
 [  3.83622186e-01   6.50311664e+00   4.38647520e+00]
 [  3.92394621e+01   6.17318473e+00   9.32889304e-01]
 [  2.00764668e+01   7.51058448e+00   5.69020980e+00]
 [  2.72715318e-04   1.13382181e-03   2.05452711e+01]
 [  9.78510721e-04   1.77759877e-03   7.17500126e+01]
 [  1.19476709e-01   7.53544387e-02   1.87572727e+01]
 [  1.29056830e-03   8.34052019e-01   2.23609389e-02]
 [  2.48817468e-03   8.74666518e-03   1.34751210e+00]
 [  6.10683916e-05   6.03162591e-07   1.63539763e+00]]
[[  6.78573363e-05   1.38043057e-05  -3.00887338e-05  -1.33046699e-05
    1.37165328e-05   1.37165328e-05   1.37165328e-05   1.37165328e-05]
 [  2.84806685e-04   5.64110458e-05  -1.28372418

BEST END ITEM:
(158, 2, 1, 10, 3, 9)
---
Phone sequence:
[(0, 0, 10), (4, 10, 21), (0, 21, 33), (2, 33, 49), (0, 49, 64), (4, 64, 85), (2, 85, 104), (0, 104, 114), (2, 114, 140), (4, 140, 152), (1, 152, 159)]
[('SUB', 1, 0), ('IB', 1, 4), ('SUB', 2, 0), ('SUB', 2, 2), ('SUB', 1, 0), ('SUB', 2, 4), ('SUB', 3, 2), ('SUB', 2, 0), ('SUB', 1, 2), ('SUB', 2, 4), ('SUB', 1, 1)]
BEST END ITEM:
(127, 2, 2, 9, 3, 9)
---
Phone sequence:
[(4, 0, 9), (0, 9, 17), (4, 17, 26), (0, 26, 48), (4, 48, 62), (0, 62, 73), (4, 73, 78), (0, 78, 94), (4, 94, 105), (2, 105, 128)]
[('SUB', 0, 4), ('SUB', 1, 0), ('SUB', 0, 4), ('SUB', 0, 0), ('SUB', 2, 4), ('SUB', 0, 0), ('SUB', 2, 4), ('SUB', 1, 0), ('SUB', 3, 4), ('SUB', 0, 2)]
BEST END ITEM:
(296, 2, 0, 10, 3, 9)
---
Phone sequence:
[(1, 0, 3), (4, 3, 30), (2, 30, 66), (0, 66, 98), (2, 98, 139), (0, 139, 151), (2, 151, 195), (0, 195, 210), (2, 210, 253), (4, 253, 273), (0, 273, 297)]
[('SUB', 0, 1), ('IB', 0, 4), ('SUB', 2, 2), ('SUB', 1, 0), ('SUB', 3, 2), ('

BEST END ITEM:
(558, 2, 2, 10, 3, 9)
---
Phone sequence:
[(0, 0, 30), (2, 30, 59), (4, 59, 112), (2, 112, 195), (0, 195, 227), (4, 227, 297), (2, 297, 390), (0, 390, 396), (2, 396, 500), (0, 500, 518), (2, 518, 559)]
[('SUB', 1, 0), ('IB', 1, 2), ('SUB', 0, 4), ('SUB', 1, 2), ('SUB', 1, 0), ('SUB', 0, 4), ('SUB', 2, 2), ('SUB', 3, 0), ('SUB', 2, 2), ('SUB', 3, 0), ('SUB', 2, 2)]
BEST END ITEM:
(141, 2, 0, 9, 3, 9)
---
Phone sequence:
[(1, 0, 4), (0, 4, 26), (4, 26, 38), (0, 38, 45), (4, 45, 60), (0, 60, 75), (4, 75, 90), (0, 90, 106), (2, 106, 130), (0, 130, 142)]
[('SUB', 1, 1), ('SUB', 1, 0), ('SUB', 1, 4), ('SUB', 0, 0), ('SUB', 2, 4), ('SUB', 3, 0), ('SUB', 2, 4), ('SUB', 2, 0), ('SUB', 2, 2), ('SUB', 3, 0)]
BEST END ITEM:
(129, 2, 1, 10, 3, 9)
---
Phone sequence:
[(1, 0, 3), (4, 3, 15), (0, 15, 26), (1, 26, 39), (0, 39, 53), (2, 53, 70), (0, 70, 78), (2, 78, 104), (0, 104, 107), (2, 107, 117), (1, 117, 130)]
[('SUB', 1, 1), ('IB', 1, 4), ('SUB', 2, 0), ('SUB', 1, 1), ('SUB', 0, 0)

BEST END ITEM:
(851, 2, 0, 10, 3, 9)
---
Phone sequence:
[(0, 0, 11), (4, 11, 86), (2, 86, 245), (0, 245, 263), (2, 263, 422), (0, 422, 450), (2, 450, 578), (4, 578, 609), (0, 609, 688), (2, 688, 837), (0, 837, 852)]
[('SUB', 1, 0), ('IB', 1, 4), ('SUB', 3, 2), ('SUB', 3, 0), ('SUB', 2, 2), ('SUB', 1, 0), ('SUB', 0, 2), ('SUB', 2, 4), ('SUB', 3, 0), ('SUB', 3, 2), ('SUB', 1, 0)]
BEST END ITEM:
(770, 2, 2, 10, 3, 9)
---
Phone sequence:
[(2, 0, 33), (4, 33, 84), (2, 84, 227), (0, 227, 234), (2, 234, 336), (0, 336, 390), (2, 390, 483), (0, 483, 546), (2, 546, 641), (0, 641, 702), (2, 702, 771)]
[('SUB', 3, 2), ('IB', 3, 4), ('SUB', 2, 2), ('SUB', 3, 0), ('SUB', 0, 2), ('SUB', 3, 0), ('SUB', 1, 2), ('SUB', 3, 0), ('SUB', 0, 2), ('SUB', 3, 0), ('SUB', 2, 2)]
BEST END ITEM:
(164, 2, 2, 10, 3, 9)
---
Phone sequence:
[(4, 0, 3), (0, 3, 17), (2, 17, 37), (0, 37, 52), (2, 52, 78), (0, 78, 85), (4, 85, 102), (0, 102, 119), (4, 119, 139), (0, 139, 153), (2, 153, 165)]
[('SUB', 1, 4), ('IB', 1, 0),

BEST END ITEM:
(339, 2, 4, 10, 3, 9)
---
Phone sequence:
[(0, 0, 8), (2, 8, 59), (4, 59, 88), (0, 88, 103), (2, 103, 163), (4, 163, 176), (0, 176, 229), (2, 229, 246), (0, 246, 272), (2, 272, 327), (4, 327, 340)]
[('SUB', 0, 0), ('IB', 0, 2), ('SUB', 1, 4), ('SUB', 3, 0), ('SUB', 2, 2), ('SUB', 0, 4), ('SUB', 0, 0), ('SUB', 1, 2), ('SUB', 3, 0), ('SUB', 2, 2), ('SUB', 2, 4)]
BEST END ITEM:
(206, 2, 1, 10, 3, 9)
---
Phone sequence:
[(0, 0, 3), (2, 3, 32), (0, 32, 42), (4, 42, 63), (0, 63, 84), (2, 84, 105), (0, 105, 128), (2, 128, 168), (0, 168, 184), (2, 184, 199), (1, 199, 207)]
[('SUB', 3, 0), ('IB', 3, 2), ('SUB', 1, 0), ('SUB', 1, 4), ('SUB', 0, 0), ('SUB', 0, 2), ('SUB', 0, 0), ('SUB', 2, 2), ('SUB', 3, 0), ('SUB', 2, 2), ('SUB', 0, 1)]
BEST END ITEM:
(170, 2, 1, 10, 3, 9)
---
Phone sequence:
[(0, 0, 3), (2, 3, 31), (0, 31, 37), (4, 37, 67), (0, 67, 72), (4, 72, 90), (0, 90, 115), (4, 115, 128), (0, 128, 152), (4, 152, 162), (1, 162, 171)]
[('SUB', 3, 0), ('IB', 3, 2), ('SUB', 1, 

BEST END ITEM:
(406, 2, 1, 10, 3, 9)
---
Phone sequence:
[(0, 0, 5), (4, 5, 50), (0, 50, 90), (2, 90, 127), (0, 127, 164), (4, 164, 213), (0, 213, 247), (2, 247, 318), (0, 318, 333), (2, 333, 403), (1, 403, 407)]
[('SUB', 3, 0), ('IB', 3, 4), ('SUB', 2, 0), ('SUB', 1, 2), ('SUB', 2, 0), ('SUB', 0, 4), ('SUB', 1, 0), ('SUB', 2, 2), ('SUB', 0, 0), ('SUB', 3, 2), ('SUB', 0, 1)]
BEST END ITEM:
(79, 2, 0, 10, 3, 9)
---
Phone sequence:
[(0, 0, 3), (4, 3, 9), (0, 9, 18), (4, 18, 24), (0, 24, 34), (4, 34, 40), (0, 40, 56), (2, 56, 63), (0, 63, 66), (2, 66, 75), (0, 75, 80)]
[('SUB', 2, 0), ('IB', 2, 4), ('SUB', 3, 0), ('SUB', 2, 4), ('SUB', 1, 0), ('SUB', 1, 4), ('SUB', 3, 0), ('SUB', 1, 2), ('SUB', 0, 0), ('SUB', 1, 2), ('SUB', 2, 0)]
BEST END ITEM:
(102, 2, 0, 9, 3, 9)
---
Phone sequence:
[(4, 0, 4), (0, 4, 16), (4, 16, 31), (0, 31, 36), (4, 36, 45), (0, 45, 56), (4, 56, 69), (0, 69, 77), (2, 77, 98), (0, 98, 103)]
[('SUB', 2, 4), ('SUB', 1, 0), ('SUB', 1, 4), ('SUB', 3, 0), ('SUB', 1, 4), (